In [1]:
import numpy as np
import pandas as pd
import pymongo
import bson

In [81]:
class MongoServer():
    credentials = None
    conn = None
    db = None
    collections = {}
    def __init__(self, run = False, credentials = "credentials.txt", db_name = "agile_data_science_group_3"):
        if run:
            if not self.connect2Mongo(credentials):
                print("Connection to server Failed.")
            
            if not self.connect2DataBase(db_name):
                print("Connection Data Base Failed.")
    """Rotine to connect to Mongo DB"""    
    def connect2Mongo(self, credentials = "credentials.txt"):
        try:
            #use your database name, user and password here:
            with open(credentials, 'r', encoding='utf-8') as f:
                [name,password,url,dbname]=f.read().splitlines()
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            self.conn = conn
            return True
        except pymongo.errors.ConnectionFailure as e:
            return False
    
    """Routine to connect to a Data Base"""
    def connect2DataBase(self, db_name = "agile_data_science_group_3"):
        try:
            self.db = self.conn[db_name]
            return True
        except:
            return False
    
    """Return the available collections in a data base"""
    def listOfCollections (self):
        return self.db.collection_names()
    
    """Donwload all the ollections from the data base"""
    def getAllCollections (self):
        collections = self.listOfCollections()
        for col in collections:
            self.getCollectionFromServer(col)
        return True
    
    
    """Routine to get one collection of the Data Base"""
    # return the collection
    def getCollectionFromServer (self, name_collection):
        if name_collection in self.db.collection_names():
            self.collections[name_collection] = self.db.get_collection(name_collection)
            print ("Collection ",name_collection," Update in Local.")
            return True
        return False
    
    """ Take the collection from the local copy"""
    def getCollection(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return self.collections[name_collection]
    
    """ Gets all the items of the collection"""
    def getItems(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return [element for element in self.collections[name_collection].find()]
    
    """Query in one Collection"""
    def searchInCollection(self, name_collection, field, patro):
        return [element for element in self.collections[name_collection].find({field:patro})]
    
    """Query in all the Collections"""
    def searchInDB (self, field, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
        return query
    
    """Find one in the collection"""
    def findOne(self, collection_name):
        return self.db.get_collection(collection_name).find_one()
    
    """Froce Brute search"""
    def searchInDBForceBrute(self, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = []
            
            self.searchInCollection(collection, )
            print(self.getItems(collection))
            for element in self.getItems(collection)[0]:
                for key in element.keys():
                    query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
                    
    def findNElement(self, collection_name, N):
        query = []
        for item in self.db.get_collection(collection_name).find().limit( N ):
            query.append(item)
        return query
    
    def insertInCollection (self, collection_name,  item):
        assert type(item) == {}, "Item must be a dictionary"
        return self.db.get_collection(collection_name).insert(item)
        
                
        
MS = MongoServer(True)   

In [55]:
class Recommender:
    
    def __init__(self):
        # connect to mongo with MongoServer object
        self.server = MongoServer(True)
        
    def dummieRecommendation(self, N = 10):
        listObjectIds = []
        for item in server.findNElement('recipes', N):
            listObjectIds.append(item['_id'])
        return listObjectIds
        
    
        
    def isObjectId(self, _id):
        try:
            # Do a query in a specific user collection
            if not type(_id) == bson.objectid.ObjectId:
                # creation of a objectID
                if type(_id) == str:
                    idUser = bson.objectid.ObjectId(_id)
                else:
                    return None
            return _id
        except:
            return None
            
    """
    Search user by idRecepie(ObjectId)
    """ 
    def searchRecepieWithIngredientsByIs(self, idRecepie):
        idRecepie = self.isObjectId(idRecepie)
        if idRecepie == None:
            print ("idRecepie is not a ObjectId")
            return []
        
        # Search the recepie    
        _collection = 'recipes_ingredients'
        _field = '_id' # ObjectId
        _patro = idRecepie
        query = self.server.searchInCollection(name_collection = _collection, field = _field, patro = _patro)[0]    
        # obtain the ingredients
        return query, query.values()[1]
    

    """
    Search user by idUser(ObjectId)
    """
    def searchUsersById(self, idUser):
        idUser = self.isObjectId(idUser)
        if idUser == None:
            print ("Id User is not a ObjectId")
            return []
        
        _collection = 'users'
        _field = '_id' # ObjectId
        _patro = idUser
        query = self.server.searchInCollection(name_collection = _collection, field = _field, patro = _patro)[0]
        
        return query
    
    def computeRecommenderMatrixBestRated(self, idRecipe):
        # look the ingredients used in the recipe
        # look the recipe that use on of the ingridients
        # compute Recommender Matrix 
        matrix = pd.DataFrame(random, index=ids, columns=user)
        return matrix
    
    def computeRecommenderMatrixCollaborativeFiltering(self, idUser):
        m_ids = []
        m_user = []
        
        idUser = self.isObjectId(idUser)
        if idUser == None:
            print ("Id User is not a ObjectId")
            return []
        # look for the ratings of the user
        ratingsUser = self.server.searchInCollection(name_collection='ratings', field='user_id', patro=idUser)
        if ratingsUser == []:
            print ("User has no ratings. Cold Start.")
            return None
        ratings = []
        m_user.append(idUser) 
        # obtain ratings from the same recipes
        for rating in ratingsUser:
            m_ids.append(rating['recipe_id'])
            # search for more ratings in the same recipe
            recipes = self.server.searchInCollection(name_collection='ratings', field='recipe_id', patro=rating['recipe_id'])
            
            # acumulate the ratings
            ratings += recipes
            
            # look if objectId of recipe is in the list
            for recipe in recipes:
                if not recipe['recipe_id'] in m_ids:
                    m_ids.append(recipe['recipe_id'])
        
        print ("Recipes ids: ", m_ids)
        print ("Users ids: ", m_user)
            
        # compute Recommender Matrix 
        matrix = pd.DataFrame(np.full((len(m_ids), len(m_user)), np.nan), index=m_ids, columns=m_user)
        for rates in ratings:
            matrix[rates['user_id']][rates['recipe_id']] = rates['rating']
        
        return matrix
    
    def generateRatingMatrix(self):
        m_ids = set()
        m_user = set()
        ratings = self.server.getItems('ratings')
        for item in ratings:
            m_ids.add(item["recipe_id"])
            m_user.add(item["user_id"])
        
        matrix = pd.DataFrame(np.full((len(m_ids), len(m_user)), np.nan), index=m_ids, columns=m_user)
        for rates in ratings:
            matrix[rates['user_id']][rates['recipe_id']] = rates['rating']
        
        return matrix
            
    def computeRecomendation(self):
        # Introduce here the distance function for each of the cases
        # maybe it is necesary to separete the funciton in two
        pass
    
    def bestRated(self, idRecepie, n):
        # take the recepie
        # look for all the recepie containing similar ingridients
        # generate the recommender matrix recepie - ingridient
        return self.dummieRecommendation(N)
    
    def collaborativeFiltering(self, idUser):
        # take the ratings of the user
        # with the recipes of the user, find which recepes we can generate
        # generate the recommender matrix for user
        # call the distance function
        return self.dummieRecommendation(N)
    
    def searchByTngredients(self, listIngredients, N = 10):
        return self.dummieRecommendation(N)
        
    
    
rec = Recommender()

#rec.computeRecommenderMatrixCollaborativeFiltering(bson.objectid.ObjectId('5a1aa2cb2cfaa80d6ff251f3'))
#rec.generateRatingMatrix()

In [36]:
rec.server.searchInCollection(name_collection='ratings', field='user_id', patro=bson.objectid.ObjectId('5a1aa9602cfaa80d6ff25314'))

[]

In [128]:
rec.server.collections['ratings'].count()

4967